# crms_irf.ipynb

trying iterative random forests on random forest generated from both 
1. try irf on housekeeping gene subset of model for each cell type


based on https://github.com/Yu-Group/iterative-Random-Forest/blob/master/demo/28_iRF_demo_sklearn.ipynb

and the Iteratve Random Forest (Basu paper)


update from original (make easier to run multiple batches)


multiple batches are adaboosted based on accuracy



In [1]:
import warnings
warnings.filterwarnings('ignore')
# basic packages
import os, glob
import pandas as pd
import numpy as np; np.random.seed(0)
import itertools
from collections import Counter, defaultdict
import time

# machine learning packages from sklearn
from sklearn.preprocessing import MinMaxScaler #StandardScaler 
from sklearn import preprocessing, metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, VotingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, auc, roc_curve, plot_roc_curve, confusion_matrix, accuracy_score
from scipy import interp
from subprocess import call
from IPython.display import Image


# for IRF
from functools import reduce
# Needed for the scikit-learn wrapper function
import irf
from irf import (irf_utils, utils,
                 irf_jupyter_utils)
from irf.ensemble.wrf import RandomForestClassifierWithWeights
from math import ceil

# Import our custom utilities
from imp import reload


# Import tools needed for visualization
import seaborn as sns; sns.set()
import matplotlib
import matplotlib.pyplot as plt
from sklearn.tree import export_graphviz
import pydot

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
save_dir = '../data/processed/fig4_modelling/irf_manual'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [4]:
THRES=1

In [5]:
normal_tissues = ['Airway','Astrocytes','Bladder','Colon','Esophageal','GDSD0','GDSD3','GDSD6','GM12878','HMEC','Melanocytes','Ovarian',
'Pancreas','Prostate','Renal','Thyroid','Uterine']
normal_tissues_dict = dict(zip(normal_tissues,range(len(normal_tissues))))

In [6]:
rna_df = pd.read_csv('../data/interim/rna/tissue_tpm_sym.csv',index_col=0)
rna_df_norm = rna_df[normal_tissues]

# 0. Data Wrangling
- import
- preprocess


In [7]:
# import
data_all = pd.read_csv('/Users/mguo123/Google Drive/1_khavari/omics_project-LD/pan_omics/data/processed/tissue_crms/all_count_comb_overall.csv',index_col=0,header=0)
data_all = data_all[data_all.tissue.isin(normal_tissues)]
data_all = data_all[data_all.iloc[:,2:].sum(axis=1)>1e-1]

# expression labels
# exp_label = list(data_all.exp.values)
labels_all  = data_all.tissue.values#np.array((data_all.exp>THRES).values)
tissue_num_labels =  data_all.tissue.map(normal_tissues_dict).values

genes_all = data_all.index.values
gene_to_num_dict = dict(zip(np.unique(genes_all),range(len(np.unique(genes_all)))))
genes_num_all = np.vectorize(gene_to_num_dict.get)(genes_all)



In [8]:
print('files_loaded', data_all.shape)


files_loaded (156071, 1063)


In [9]:
data_all[:5]

,tissue,exp,num_loop_counts,num_loops,num_atac_regions_pro,num_atac_regions_loop,AHR_pro,ARID5B_pro,ARNT_pro,ARNTL_pro,...,MAFA_loop,MEOX1_loop,E2F5_pro,E2F5_loop,ESR2_pro,KLF14_pro,TBX15_pro,ESR2_loop,KLF14_loop,TBX15_loop
A4GALT,Airway,5.435,45.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A4GNT,Airway,0.000,5060.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAMP,Airway,43.710,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AANAT,Airway,0.000,13.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ABAT,Airway,0.125,21.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
## only tfs
for feature in ['tissue','exp','num_loop_counts','num_loops','num_atac_regions_pro','num_atac_regions_loop']:
    if feature in data_all.columns:
        data_all.drop(feature,axis=1,inplace=True)

data_all.shape




(156071, 1057)

In [11]:


selector = VarianceThreshold()
data_all_varfilt = selector.fit_transform(data_all)
data_all_varfilt_cols = data_all.columns[selector.get_support()]
print(data_all.shape, data_all_varfilt.shape, len(data_all_varfilt_cols))
scaler = MinMaxScaler()
data_all_norm = scaler.fit_transform(data_all_varfilt)
data_all_norm = pd.DataFrame(data_all_norm, columns = data_all_varfilt_cols)

(156071, 1057) (156071, 984) 984


In [12]:
data_all_norm[:5]

,AHR_pro,ARID5B_pro,ARNT_pro,ARNTL_pro,ATF1_pro,ATF2_pro,ATF3_pro,ATF4_pro,ATF6_pro,ATF7_pro,...,TFAP2B_pro,ZNF382_pro,MYOD1_loop,NR4A3_loop,TFAP2B_loop,ZNF382_loop,CUX2_pro,CUX2_loop,E2F5_pro,E2F5_loop
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 2. get gene sets

create dictionary {tissue: gene_list} for each category

2A. unique gene lists for each cell type

In [13]:
glob.glob('../../rnaseq/unique_gene_lists/*genes.txt')

['../../rnaseq/unique_gene_lists/Colon_genes.txt',
 '../../rnaseq/unique_gene_lists/GDSD0_genes.txt',
 '../../rnaseq/unique_gene_lists/Airway_genes.txt',
 '../../rnaseq/unique_gene_lists/GDSD6_genes.txt',
 '../../rnaseq/unique_gene_lists/Pancreas_genes.txt',
 '../../rnaseq/unique_gene_lists/Uterine_genes.txt',
 '../../rnaseq/unique_gene_lists/Ovarian_genes.txt',
 '../../rnaseq/unique_gene_lists/Astrocytes_genes.txt',
 '../../rnaseq/unique_gene_lists/Bladder_genes.txt',
 '../../rnaseq/unique_gene_lists/Melanocytes_genes.txt',
 '../../rnaseq/unique_gene_lists/HMEC_genes.txt',
 '../../rnaseq/unique_gene_lists/Prostate_genes.txt',
 '../../rnaseq/unique_gene_lists/GM12878_genes.txt',
 '../../rnaseq/unique_gene_lists/Renal_genes.txt',
 '../../rnaseq/unique_gene_lists/GDSD3_genes.txt',
 '../../rnaseq/unique_gene_lists/Esophageal_genes.txt',
 '../../rnaseq/unique_gene_lists/Thyroid_genes.txt']

In [14]:
unique_genesets = {}
for rna_file in glob.glob('../../rnaseq/unique_gene_lists/*genes.txt'):
    tissue = os.path.basename(rna_file).split('_')[0]
    geneset = sorted(pd.read_csv(rna_file,header=None).loc[:,0].dropna().unique())
    print(tissue, len(geneset))
    unique_genesets[tissue] = geneset
    

Colon 14
GDSD0 14
Airway 41
GDSD6 240
Pancreas 17
Uterine 37
Ovarian 27
Astrocytes 465
Bladder 37
Melanocytes 226
HMEC 49
Prostate 312
GM12878 447
Renal 27
GDSD3 61
Esophageal 19
Thyroid 44


2B. expressed gene lists for each cell type

In [15]:
THRES=5

In [16]:
expr_genesets = {}
for tissue in rna_df_norm.columns:
    geneset = list(rna_df_norm[tissue][rna_df_norm[tissue]>THRES].index)
    print(tissue, len(geneset))
    expr_genesets[tissue] = geneset

Airway 6815
Astrocytes 7303
Bladder 6642
Colon 6826
Esophageal 6651
GDSD0 7031
GDSD3 6970
GDSD6 6442
GM12878 6166
HMEC 6972
Melanocytes 7359
Ovarian 6578
Pancreas 7055
Prostate 6516
Renal 6901
Thyroid 7003
Uterine 6569


2C. group genesets

In [17]:
print(glob.glob('../data/interim/rna/*_genes.csv'))
group_tissue_mapping = {
    'blue':['Astrocytes','Melanocytes'],
    'grey':['GM12878'],
    'green':['Colon','Esophageal','Ovarian','Pancreas','Renal','Thyroid'],
    'purple':['Airway','Bladder', 'GDSD6', 'HMEC', 'Prostate', 'Uterine'] 
}

['../data/interim/rna/purple_genes.csv', '../data/interim/rna/green_genes.csv', '../data/interim/rna/common_genes.csv', '../data/interim/rna/all_genes.csv', '../data/interim/rna/grey_genes.csv', '../data/interim/rna/blue_genes.csv']


In [18]:
group_genesets = {}
for group, tissues in group_tissue_mapping.items():
    geneset = sorted(pd.read_csv('../data/interim/rna/'+group+'_genes.csv',header=None).loc[:,0].dropna().unique())
    for tissue in tissues:
        group_genesets[tissue] = geneset
        print(tissue, len(geneset))
        

Astrocytes 954
Melanocytes 954
GM12878 626
Colon 759
Esophageal 759
Ovarian 759
Pancreas 759
Renal 759
Thyroid 759
Airway 796
Bladder 796
GDSD6 796
HMEC 796
Prostate 796
Uterine 796


# A. Problem 1: try irf on housekeeping gene subset of model for each cell type

try GDSD3

## A.1 split data and run RF



future gene sets to test,
- housekeeping genes
- high expression genes
- all genes

In [14]:
# # get housekeeping genes
# housekeeping_df = pd.read_table(os.path.join('../../annon', 'HSIAO_HOUSEKEEPING_GENES.txt'),skiprows=2, header=None)
# housekeeping_df.columns = ['genes']
# housekeeping_genes_list = list(housekeeping_df.genes)
# len(housekeeping_genes_list)

# # tss_df_tissue = pd.read_csv('atac_footprinting/tss_footprints/GDSD3_tss_hoco_tf_exp_1000.csv', index_col=0)



In [15]:
# rf_tissue = 'Prostate'
# gene_cluster = 'Prostate' # green, grey, purple, blue
# tissue_specific = True
# negative_subsample_factor = 1 # factor = #neg/#pos

In [23]:
def get_genes(gene_cluster,tissue_specific='unique'):
    if tissue_specific=='unique':
        cell_type_genes = unique_genesets[gene_cluster]        
    elif tissue_specific=='expr':
        cell_type_genes = expr_genesets[gene_cluster]
    elif tissue_specific =='group':
        cell_type_genes = group_genesets[gene_cluster]
    else:
        raise ValueError('geneset dictionary not found')
    
    print('num genes looking at', len(cell_type_genes))

    genes_bool = np.isin(genes_all, cell_type_genes)
    genes_bool_idx = np.where(genes_bool)[0]
    print('loaded genes group')
    print('num genes samples considered', len(genes_bool))
    print('num genes samples with selected genes chosen',len(genes_bool_idx))
    return genes_bool, genes_bool_idx 

In [24]:
def run_irf(rf_tissue, gene_cluster, tissue_specific='unique', num_iter=5, boost=True, negative_subsample_factor=1):
    ## get genes
    print('######## 1. get samples ############')
    ### get tissue sample idx
    tissue_bool_labels = (labels_all==rf_tissue)
    tissue_bool_idx = np.where(tissue_bool_labels)[0]
    print('number samples that tissue has', tissue_bool_idx.shape)

    ### get gene sample idx# option 1: use tpm1 cell type specific genes/    
    genes_bool, genes_bool_idx = get_genes(gene_cluster,tissue_specific)
    

    print('######## 2. iterate through negative subsampling ############')
    stability_df_overall = pd.DataFrame()
    acc_arr = []
    for idx in range(num_iter):
        print(idx, '######## 2A. iterate through negative subsampling ############')
        # negative subsample: genes of interest but not tissue of interest
        # positive = genes of interest and tissue of interest
        pos_idx  = set(tissue_bool_idx).intersection(genes_bool_idx)
        print('number tissue samples chosen', len(pos_idx))
        neg_idx = set(genes_bool_idx) - set(tissue_bool_idx)
        print('number negative samples pre-subsampled', len(neg_idx))
        neg_idx_subsampled = np.random.choice(list(neg_idx), len(pos_idx)*negative_subsample_factor, replace=False)
        print('number negative samples post-subsampled', len(neg_idx_subsampled))

        select_idx = np.array(list(set(neg_idx_subsampled).union(pos_idx)))
        print('number selected samples', len(select_idx))
        print('total samples in population', data_all_norm.shape[0])
        print('% data chosen', len(select_idx)/data_all_norm.shape[0])
        
        
        
        # only include features that have positive directionality 
        feat_pos_bool = np.array(data_all.iloc[np.array(list(pos_idx)),:][data_all_varfilt_cols].sum(axis=0))*negative_subsample_factor > np.array(data_all.iloc[np.array(list(neg_idx_subsampled)),:][data_all_varfilt_cols].sum(axis=0))
        feat_pos_bool_idx = np.where(feat_pos_bool)[0]
        print('num features in positive direction', len(feat_pos_bool_idx))
        print(idx, '######## 2B. run model ############')
        # run model
        train_features, test_features, train_labels, test_labels = train_test_split(np.array(data_all_norm)[select_idx,:][:,feat_pos_bool_idx], 
                                                                                  tissue_bool_labels[select_idx], 
                                                                                  test_size = 0.25, random_state = 42)

        # Create the model with 100 trees
        model = RandomForestClassifier(n_estimators=100,#50, 
                                     max_features = 'sqrt',
                                     max_depth=5,
                                     n_jobs=-1, verbose = 0)
        # Fit on training data
        model.fit(train_features, train_labels)
        # Actual class predictions
        rf_predictions = model.predict(test_features)
        # evaluation of model
        print(confusion_matrix(test_labels, rf_predictions))
        acc = accuracy_score(test_labels, rf_predictions)
        acc_arr.append(acc)
        print('acc', acc)

        print(idx, '######## 2C. get IRF ############')
        try:
            # irf start 
            # 1. get rf data
            all_rf_tree_data = utils.get_rf_tree_data(
              rf=model, X_train=train_features, X_test=test_features, y_test=test_labels)
            # 2. get rit data
            all_rit_tree_data = irf_utils.get_rit_tree_data(
              all_rf_tree_data=all_rf_tree_data,
              bin_class_type=1,
              M=100,
              max_depth=2,
              noisy_split=False,
              num_splits=2)
            # 3. run irf
            all_rf_weights, all_K_iter_rf_data, \
            all_rf_bootstrap_output, all_rit_bootstrap_output, \
            stability_score = irf_utils.run_iRF(X_train=train_features,
                                              X_test=test_features,
                                              y_train=train_labels,
                                              y_test=test_labels,
                                              K=10,
                                              rf=RandomForestClassifierWithWeights(n_estimators=100),
                                              B=30,
                                              random_state_classifier=2018,
                                              propn_n_samples=.2,
                                              bin_class_type=1,
                                              M=25,
                                              max_depth=5,
                                              noisy_split=False,
                                              num_splits=2,
                                              n_estimators_bootstrap=5)
            # 4. get stability score
            stability_score_names = {}
            for feat_idx, score in sorted(stability_score.items(),key=lambda x:x[1],reverse=True):
                feat_names_arr = [data_all_varfilt_cols[int(x)] for x in feat_idx.split('_')]
                feat_names = '::'.join(feat_names_arr)
                stability_score_names[feat_names] = score
            stability_df = pd.DataFrame(pd.Series(stability_score_names),columns=['score']).reset_index()
            stability_df['run'] = idx
            stability_df['acc'] = acc
            stability_df_overall = pd.concat([stability_df_overall, stability_df], axis=0)
        except:
            print('irf run failed', num_iter)
    
    print('######## 3. get stats and save ############')
    if stability_df_overall.shape[0]>0:
        # stats and save
        save_file = os.path.join(save_dir, 'test_'+rf_tissue+'_' + gene_cluster + '_stability_score.csv')

        stability_df_agg = stability_df_overall.groupby('index').agg({'score':['mean','std','count']})
        stability_df_agg.columns = ['score', 'std', 'count']
        stability_df_agg.sort_values('score',ascending=False)
        stability_df_agg.to_csv(save_file)
        print('saved', save_file)

        # for boosting
        if boost:
            # for reweighting
            acc_arr_norm = np.array(acc_arr)/max(acc_arr)
            acc_arr_norm_sum = sum(acc_arr_norm)
            boost_factor = len(acc_arr_norm)/acc_arr_norm_sum
            
            save_file = os.path.join(save_dir, 'test_'+rf_tissue+'_' + gene_cluster + '_boosted_stability_score.csv')
            stability_df_overall['norm_acc'] = stability_df_overall['acc']/max(stability_df_overall['acc'])*boost_factor
            stability_df_overall['boost_score'] = stability_df_overall['score']*stability_df_overall['norm_acc']
            stability_df_agg = stability_df_overall.groupby('index').agg({'boost_score':['mean','std','count']})
            stability_df_agg.columns = ['score', 'std', 'count']
            stability_df_agg.sort_values('score',ascending=False)
            stability_df_agg.to_csv(save_file)
            print('saved boosted file', save_file)
        return stability_df_agg#, all_rf_tree_data#test_labels, test_features
    else:
        print('no results')
        return 

In [22]:
%%time
### TESTING ###
df  = run_irf('GDSD6', 'GDSD6', tissue_specific='unique',num_iter=5, boost=True)

######## 1. get samples ############
number samples that tissue has (8812,)
num genes looking at 240
loaded genes group
num genes samples considered 156071
num genes samples with selected genes chosen 1013
######## 2. iterate through negative subsampling ############
0 ######## 2A. iterate through negative subsampling ############
number tissue samples chosen 79
number negative samples pre-subsampled 934
number negative samples post-subsampled 79
number selected samples 158
total samples in population 156071
% data chosen 0.0010123597593403003
num features in positive direction 300
0 ######## 2B. run model ############
[[13  4]
 [ 8 15]]
acc 0.7
0 ######## 2C. get IRF ############
irf run failed 5
1 ######## 2A. iterate through negative subsampling ############
number tissue samples chosen 79
number negative samples pre-subsampled 934
number negative samples post-subsampled 79
number selected samples 158
total samples in population 156071
% data chosen 0.0010123597593403003
num feature

In [105]:
# np.array(data_all_norm)[select_idx,:][:,feat_pos_bool_idx].shape

In [106]:
# all_rf_tree_data
# np.array(data_all.iloc[np.array(list(pos_idx)),:][data_all_varfilt].sum(axis=0)

In [118]:
# feat_pos_bool = np.array(data_all.iloc[np.array(list(pos_idx)),:][data_all_varfilt_cols].sum(axis=0)) > np.array(data_all.iloc[np.array(list(neg_idx_subsampled)),:][data_all_varfilt_cols].sum(axis=0))
# feat_pos_bool_idx = np.where(feat_pos_bool)[0]
# print(list(data_all_varfilt_cols[feat_pos_bool_idx]))

In [133]:
for x in data_all.iloc[np.array(list(pos_idx)),:].index:
    print(x)

YPEL4
TRIM17
NPAS1
IGSF22
L3MBTL4
PHYHIPL
TCHH
FAM156A
IL1F10
FAM180A
SAPCD1
GPLD1
LCE1D
TET3
TEX12
LCE3C
LCE4A
LCE5A
TFE3
MORN3
ZNF117
PLA2G2F
PLA2G4E
LGI2
IPPK
THEM5
LIPK
LIPM
ZNF416
ZNF460
SPDYA
PNPLA1
ZNF616
DUSP8
COX8C
CCL27
CPEB3
SPRR2B
FKBPL
FLG2
ABHD12B
ABHD8
KCNJ11
TMEM170B
EFHB
SHE
ACER1
USP2
ATP10B
ATP13A4
HMOX1
SLC10A6
CTNS
SLC13A4
SLC15A1
ELF5
ELMOD1
TMPRSS12
TMPRSS13
RNF223
HRNR
CYP26B1
GAST
SUN3
SLC28A3
HSPA1B
ALDH1A2
PSG2
PSG6
ALDH8A1
PDZD2
KRT2
KRT24
ESYT3
GJA9
XKRX
RPTN
SLC5A1
KRT79


In [137]:
df[df['count']>1].sort_values('score',ascending=False)[:50]

,score,std,count
index,,,
ELK1_pro,0.871522,0.127271,2
FOSB_pro,0.725281,0.020933,2
FOXO3_pro,0.692718,0.024867,3
ETV3_pro,0.624038,0.249518,2
E2F8_pro,0.510361,0.208599,3
E2F1_pro,0.482534,0.126569,4
DDIT3_pro,0.472074,0.367504,3
ELF2_pro,0.470101,0.140593,3
FOS_pro,0.442866,0.529179,2


In [123]:
# feature_importances_rank_idx = all_rf_tree_data['feature_importances_rank_idx']
# feature_importances = all_rf_tree_data['feature_importances']

# pd.DataFrame({#'rank': range(1,train_features.shape[1]+1),
#              'feature': data_all_varfilt_cols[feature_importances_rank_idx],
#              'score':feature_importances[feature_importances_rank_idx]})

In [193]:
# %%time
# run_irf('Prostate', 'purple', tissue_specific='group')

# run through all tissue specific models

In [124]:
%%time
for tissue in ['Airway','Astrocytes','Bladder','Colon','Esophageal','GDSD6','GM12878','HMEC','Melanocytes','Ovarian',
'Pancreas','Prostate','Renal','Thyroid','Uterine']:
    print('***************', tissue)
    run_irf(tissue, tissue, num_iter=5, tissue_specific='unique', boost=True)
    

*************** Airway
######## 1. get samples ############
number samples that tissue has (5771,)
num genes looking at 42
loaded genes tissue
num genes samples considered 156071
num genes samples with selected genes chosen 138
######## 2. iterate through negative subsampling ############
######## 2A. iterate through negative subsampling ############
number tissue samples chosen 6
number negative samples pre-subsampled 132
number negative samples post-subsampled 6
number selected samples 12
total samples in population 156071
% data chosen 7.68880829878709e-05
num features in positive direction 100
######## 2B. run model ############
[[1 0]
 [2 0]]
acc 0.3333333333333333
######## 2C. get IRF ############
irf run failed 5
######## 2A. iterate through negative subsampling ############
number tissue samples chosen 6
number negative samples pre-subsampled 132
number negative samples post-subsampled 6
number selected samples 12
total samples in population 156071
% data chosen 7.6888082987870

In [125]:
os.listdir('../../rnaseq/unique_gene_lists/')

['Colon_genes.txt',
 'GDSD0_genes.txt',
 'Airway_genes.txt',
 'Icon\r',
 'GDSD6_genes.txt',
 'Pancreas_genes.txt',
 'Uterine_genes.txt',
 'Ovarian_genes.txt',
 'Astrocytes_genes.txt',
 'Bladder_genes.txt',
 'Melanocytes_genes.txt',
 'HMEC_genes.txt',
 'Prostate_genes.txt',
 'GM12878_genes.txt',
 'all_tissue_specific_genes.csv',
 '.ipynb_checkpoints',
 'Renal_genes.txt',
 'GDSD3_genes.txt',
 'Esophageal_genes.txt',
 'Thyroid_genes.txt']

In [138]:
groups_dict = {'Airway':'purple',
 'Astrocytes':'blue',
 'Bladder':'purple',
 'Colon':'green',
 'Esophageal':'green',
 'GDSD0':'purple',
 'GDSD3':'purple',
 'GDSD6':'purple',
 'GM12878':'grey',
 'HMEC':'purple',
 'Melanocytes':'blue',
 'Ovarian':'green',
 'Pancreas':'green',
 'Prostate':'purple',
 'Renal':'green',
 'Thyroid':'green',
 'Uterine':'purple'}


In [139]:
%%time
for tissue,group in groups_dict.items():
    print('***************', tissue, group)
    run_irf(tissue, group, num_iter=5, tissue_specific='group', boost=True)
    

*************** Airway purple
######## 1. get samples ############
number samples that tissue has (5771,)
loaded genes group
num genes samples considered 156071
num genes samples with selected genes chosen 6315
######## 2. iterate through negative subsampling ############
######## 2A. iterate through negative subsampling ############
number tissue samples chosen 316
number negative samples pre-subsampled 5999
number negative samples post-subsampled 316
number selected samples 632
total samples in population 156071
% data chosen 0.004049439037361201
num features in positive direction 233
######## 2B. run model ############
[[74  3]
 [39 42]]
acc 0.7341772151898734
######## 2C. get IRF ############
######## 2A. iterate through negative subsampling ############
number tissue samples chosen 316
number negative samples pre-subsampled 5999
number negative samples post-subsampled 316
number selected samples 632
total samples in population 156071
% data chosen 0.004049439037361201
num features

update 08/14/2020

run with larger expression dataset

(takes a while to run so try on sherlock?)


In [25]:
%%time
# 08/14/2020
for tissue in ['Colon','GDSD6','GM12878','Melanocytes']:
    print('***************', tissue)
    run_irf(tissue, tissue, num_iter=5, tissue_specific='expr', boost=True)
    

*************** Colon
######## 1. get samples ############
number samples that tissue has (10518,)
num genes looking at 6826
loaded genes group
num genes samples considered 156071
num genes samples with selected genes chosen 66936
######## 2. iterate through negative subsampling ############
0 ######## 2A. iterate through negative subsampling ############
number tissue samples chosen 4658
number negative samples pre-subsampled 62278
number negative samples post-subsampled 4658
number selected samples 9316
total samples in population 156071
% data chosen 0.059690781759583776
num features in positive direction 467
0 ######## 2B. run model ############
[[1078   90]
 [ 335  826]]
acc 0.8175182481751825
0 ######## 2C. get IRF ############
irf run failed 5
1 ######## 2A. iterate through negative subsampling ############
number tissue samples chosen 4658
number negative samples pre-subsampled 62278
number negative samples post-subsampled 4658
number selected samples 9316
total samples in popu

KeyboardInterrupt: 

ran on sherlock.
